In [102]:
# Linear Algebra
import numpy as np

# Data Processing
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt
import time
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# Algorithm 

from sklearn.svm import SVC, LinearSVC

In [127]:
import io
start_time = time.time()
kidney_data = pd.read_csv('newdataset.csv')
kidney_data.drop(['Unnamed: 0'],axis=1,inplace=True)
kidney_data.loc[:7,['age','stages']]
kidney_data.iloc[:7,0:24]
#kidney_data.info()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,stages
0,48,80.0,1.020,1.0,0.0,1,1,0,0,121.0,...,32,72,31,1,4,1,0,0,0,2
1,7,50.0,1.020,4.0,0.0,1,1,0,0,121.0,...,26,56,31,0,3,1,0,0,0,1
2,62,80.0,1.010,2.0,3.0,1,1,0,0,423.0,...,19,70,31,0,4,1,1,0,1,3
3,48,70.0,1.005,4.0,0.0,1,0,1,0,117.0,...,20,62,18,1,3,1,1,1,1,4
4,51,80.0,1.010,2.0,0.0,1,1,0,0,106.0,...,23,68,25,0,3,1,0,0,0,3
5,60,90.0,1.015,3.0,0.0,1,1,0,0,74.0,...,27,72,23,1,4,1,0,1,0,2
6,68,70.0,1.010,0.0,0.0,1,1,0,0,100.0,...,24,72,23,0,3,1,0,0,0,5


In [113]:
kidney_data['pc'].unique()

array([1, 0], dtype=int64)

In [135]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
kidney_data['age'].values
X=minmax.fit_transform(kidney_data.iloc[:,0:24].to_numpy())
asa=minmax.fit_transform(kidney_data.iloc[:,24:25].to_numpy())
asa

array([0.25])

In [105]:
kidney_data[375:,0]

TypeError: '(slice(375, None, None), 0)' is an invalid key

In [107]:
#kidney_data = kidney_data.sample(frac = 1)
kidney_data=kidney_data.to_numpy()


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [110]:
kidney_data[0:3]

array([[ 48.  ,  80.  ,   1.02,   1.  ,   0.  ,   1.  ,   1.  ,   0.  ,
          0.  , 121.  ,  36.  ,   1.2 , 111.  ,   2.5 ,  15.4 ,  32.  ,
         72.  ,  31.  ,   1.  ,   4.  ,   1.  ,   0.  ,   0.  ,   0.  ,
          2.  ],
       [  7.  ,  50.  ,   1.02,   4.  ,   0.  ,   1.  ,   1.  ,   0.  ,
          0.  , 121.  ,  18.  ,   0.8 , 111.  ,   2.5 ,  11.3 ,  26.  ,
         56.  ,  31.  ,   0.  ,   3.  ,   1.  ,   0.  ,   0.  ,   0.  ,
          1.  ],
       [ 62.  ,  80.  ,   1.01,   2.  ,   3.  ,   1.  ,   1.  ,   0.  ,
          0.  , 423.  ,  53.  ,   1.8 , 111.  ,   2.5 ,   9.6 ,  19.  ,
         70.  ,  31.  ,   0.  ,   4.  ,   1.  ,   1.  ,   0.  ,   1.  ,
          3.  ]])

In [86]:
#x_train = kidney_data[:,1:]
x_test = kidney_data.iloc[375:,1:]
y_train = kidney_data[:,0:]
y_test = kidney_data.iloc[375:,25:26]

TypeError: '(slice(None, None, None), slice(0, None, None))' is an invalid key

In [99]:
y_test = kidney_data.iloc[375:,24:25]
y_test.info()
l=y_test.iloc[5]
l

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 375 to 399
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   stages  25 non-null     int64
dtypes: int64(1)
memory usage: 332.0 bytes


stages    2
Name: 380, dtype: int64

In [44]:
x_train

array([[80.   ,  1.02 ,  1.   , ...,  0.   ,  0.   ,  2.   ],
       [50.   ,  1.02 ,  4.   , ...,  0.   ,  0.   ,  1.   ],
       [80.   ,  1.01 ,  2.   , ...,  0.   ,  1.   ,  3.   ],
       ...,
       [80.   ,  1.02 ,  0.   , ...,  0.   ,  0.   ,  1.   ],
       [60.   ,  1.025,  0.   , ...,  0.   ,  0.   ,  1.   ],
       [80.   ,  1.025,  0.   , ...,  0.   ,  0.   ,  2.   ]])

In [45]:
def pnn_classifier(example) :
    
    summation = [None] * 5
    class_observations = 0
    for classi in range(5) :
        summation[classi] = 0.0
        class_observations=0
        for observation in range(0,385) :
            product = 0.0
            if y_train[observation] == classi :
                class_observations += 1
                for feature in range(0,24) :
                    if feature == 0:
                        product += pow(((example[feature])-x_train[observation][feature]),3)
                    elif feature == 11:
                        product += pow(((example[feature])-x_train[observation][feature]),4)
                    else :
                        product += pow((example[feature]-x_train[observation][feature]),2)
                        
                try:
                    summation[classi] += math.exp(-(math.sqrt(product)/(2.0*pow(0.0292,2))))
                except:
                    pass
        
        summation[classi]=(summation[classi]/class_observations)
                
    return winner_takes_all(summation)

In [46]:
def main() :
    
    time.sleep(1)
    t3=time.time()
    count = 0
    for i in range(0,24) :
        a = pnn_classifier(x_test[i])
        if a == y_test[i] :
            count += 1
    accuracy = count/24
    print("Accuracy:",accuracy)
    
    
    print("--- %s seconds ---" % (time.time() - t3))
    print("--- %s seconds ---" % (time.time() - start_time))

In [47]:
def winner_takes_all(summation) :
    maxval = summation[0]
    best = 0
    for classi in range(5) :
        if summation[classi] > maxval :
            best = classi
            maxval = summation[classi]
    return best

In [48]:
main()

Accuracy: 0.0
--- 0.07168269157409668 seconds ---
--- 26.071677207946777 seconds ---


In [131]:
kidney_data.iloc[:,24:25]

,stages
0,2
1,1
2,3
3,4
4,3
...,...
395,1
396,2
397,1
398,1


In [138]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()

X=minmax.fit_transform(kidney_data.iloc[:,0:24].to_numpy())

X=kidney_data.iloc[:,0:24].to_numpy()
y=kidney_data.iloc[:,24:25].to_numpy()

# Load your dataset (replace 'X' and 'y' with your actual data)
# X should be your feature matrix, and y should be the target variable.
# Make sure X is a NumPy array and y is a 1D array.

# Assuming you have X and y loaded, let's split the data into training and testing sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a PNN (here, we use KNeighborsClassifier with n_neighbors=1, which is a simple PNN)
pnn = KNeighborsClassifier(n_neighbors=1, weights='distance')
pnn.fit(X_train, y_train)

# Predict using the PNN on the test set
y_pred = pnn.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 47.50%


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load your dataset (replace 'X' and 'y' with your actual data)
# X should be your feature matrix, and y should be the target variable.
# Make sure X is a NumPy array and y is a 1D array.

# Assuming you have X and y loaded, let's split the data into training and testing sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a PNN (here, we use KNeighborsClassifier with n_neighbors=1, which is a simple PNN)
pnn = KNeighborsClassifier(n_neighbors=1, weights='distance')
pnn.fit(X_train, y_train)

# Predict using the PNN on the test set
y_pred = pnn.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
